# For Extracting Census Data from the Census API and the Census REST Server.

## Notebook to assist in the process of redistricting for Kent County. Right now the data is going to be based on the Census ACS Survery. It is still unclear if the Offical Census Data will be avaiable before the process needs to be completed.

In [ ]:
from dotenv import find_dotenv, load_dotenv
import os
from arcgis import GIS

load_dotenv(find_dotenv())

gis_user = os.getenv('ESRI_USERNAME')
gis_pass = os.getenv('ESRI_PASSWORD')
gis_url = os.getenv('PORTAL_URL')
agol_user = os.getenv('AGOL_USERNAME')
agol_pass = os.getenv('AGOL_PASSWORD')
agol_url = os.getenv('AGOL_SITE')

gisE = GIS(url=gis_url, username=gis_user, password=gis_pass)

gisA = GIS(url=agol_url, username=agol_user, password=agol_pass)

gis = GIS()

### Get Census Data for Kent County from ACS 5 Year Survey. Data is going to be at the block group level instead of the block level. 

In [ ]:
import pandas as pd
import numpy as np
import requests

census_key = os.getenv('CENSUS_KEY')

cvar = {
    'GEO_ID':'GEOID',
    'B01001_001E':'est_total',
    'B01001_001EA':'est_total_anno',
    'B01001_001M':'moe',
    'B01001_001MA':'moe_anno',
    'NAME':'name'
}

# Request parameters
payload = {
    'get' : ','.join(list(cvar.keys())),
    'for' : 'block group:*',
    'in' : ['state:10', 'county:001'],
    'key' : census_key
}

census_url = f'https://api.census.gov/data/2019/acs/acs5'

res = requests.get(census_url, params=payload)

if res.status_code == 200:
    # print(response.url)
    data = res.json()
    # print(data[0])

    
    c_df = pd.DataFrame(data[1:], columns=data[0])
    c_df.rename(columns=cvar, inplace=True)

In [ ]:
c_df.loc[:, 'GEOID'] = c_df.GEOID.str[9:]

c_df.drop(columns=['state', 'county', 'tract', 'block group'], inplace=True)

c_df.sort_values(by='GEOID', inplace=True, ignore_index=True)

census_dtypes = {
    'GEOID':'string',
    'name':'string',
    'est_total':'int32',
    'est_total_anno':'string',
    'moe':'int32',
    'moe_anno':'string'
}

c_df = c_df[list(census_dtypes.keys())].astype(census_dtypes)


In [ ]:
c_df.fillna('', inplace=True)
c_df.isna().sum()

### Get Census Block Group Geographic Data from Census REST Server. For this example, I am using ArcGIS API for Python, but you can use Open Source libraries like Geopandas.

In [ ]:
from arcgis.features import FeatureLayer

fl = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Tracts_Blocks/MapServer/8'

expr = 'STATE = 10 AND COUNTY = 01'
cbg_df = FeatureLayer(fl).query(out_sr= 4326, where=expr, out_fields='GEOID',as_df=True)

cbg_df.sort_values(by='GEOID', inplace=True, ignore_index=True)
cbg_df


<b> Method using GeoPandas instead of ArcGIS Python API. </b>

In [ ]:
import geopandas as gpd

blocks_url = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Tracts_Blocks/MapServer/8/query?'

blocks_params = {
    'where' : 'STATE = 10 and COUNTY = 001',
    'outFields' : 'GEOID',
    'f' : 'GeoJSON'
}

blocks = requests.get(blocks_url, blocks_params)

cbg_gdf = gpd.read_file(blocks.text)



cbg_gdf.sort_values(by='GEOID', inplace=True, ignore_index=True)
cbg_gdf

### Merge or join the Census API Data to Census Spatial Data 

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl

cbg_gdf.plot()

In [ ]:
print(cbg_df.columns)
print(cbg_gdf.columns)

In [ ]:
kc_gdf = cbg_gdf[['geometry']].join(c_df)
kc_df = cbg_df[['SHAPE']].join(c_df)

In [ ]:
# Define figure
fig, ax = plt.subplots(1,1)

# Plot dataframe to figure
div = make_axes_locatable(ax)
cax = div.append_axes("bottom", size='5%', pad=0.1)
kc_gdf.plot(column='est_total', legend=True, ax=ax, legend_kwds={'orientation':'horizontal'}, cax=cax, edgecolor='black', linewidth=0.4, cmap='BrBG')

# Adjust figure details
ax.set_title('Estimated Population Change from 2010-2014 to 2015-2019')
ax.axes.xaxis.set_visible(False); ax.axes.yaxis.set_visible(False)
ax.set_frame_on(False)
fig.set_figwidth(10); fig.set_figheight(10)

plt.show()

## Get Geographic Data from Census REST Server

In [ ]:
https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Tracts_Blocks/MapServer